In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_bvp

# Review: Reduction of Order

Let's review the reduction of order technique we learned in last lecture.

## Example

$$ \frac{d^2y}{dt^2} = f(t) $$
$$ y(0) = 1, y'(0) = 0 $$

Step 1: Introduce substitution variable
    
$$ \frac{dy}{dt} = z(t) $$

Step 2: Re-write equation of interest in terms of substitution variable
    
$$ \frac{d}{dt} \frac{dy}{dt} = \frac{dz}{dt} = f(t) $$

Step 3: Re-write boundary conditions

$$ y(0) = 1, z(0) = 0 $$

Step 4: Combine the results of each previous step
    
$$ \frac{dy}{dt} = z(t) $$
$$ \frac{dz}{dt} = f(t) $$
$$ y(0) = 1, z(0) = 0 $$

Step 5: Modify variable names (for convenience only)
    
$$ \frac{dy_1}{dt} = y_2(t) $$
$$ \frac{dy_2}{dt} = f(t) $$
$$ y_1(0) = 1, y_2(0) = 0 $$

In [ ]:
from scipy.integrate import solve_ivp

# let f(x) = c = 1.0
def f(t, y):
    return [y[1], 1.0]

t_span = [0., 10.]
y0 = [1.0, 0.0]

sol = solve_ivp(f, t_span, y0, t_eval=np.linspace(*t_span, 100))
y = sol.y[0]
t = sol.t
plt.plot(t, y)
plt.xlabel("t")
plt.ylabel("y")

## Exercise

Construct a system of first-order ordinary differential equations for the following equations:

$$ \frac{d^2y}{dt^2} = sin(t) $$
$$ y(0) = 1, y'(0) = 0 $$
    
$$ \frac{d^2y}{dt^2} + k^2 y = 0 $$
$$ y(0) = 1, y'(0) = 0 $$

$$ \frac{d^2y}{dt^2} + \frac{dy}{dt} + 3y = 0 $$
$$ y(0) = 1, y'(0) = 0 $$

# Boundary Value Problems

A boundary value problem is a differential equation that is solved subject to a set of constraints applied at the domain boundaries called boundary conditions. We can use `scipy.integrate.solve_bvp` to produce numerical solutions to these equations.

## Example: Simple BVP

$$ \frac{du}{dx} = f(x, u) = 1 $$
$$ u(0) = 1, 0<x<1 $$

In [ ]:
def f(x, u):
    retval = np.ones((1, x.size))
    print("retval.shape: {}".format(retval.shape))
    return retval

def bc(ua, ub):
    print("ua.shape: {}, ub.shape: {}".format(ua.shape, ub.shape))
    return [ua[0]-1]

x = np.linspace(0., 1.)
u = np.zeros((1, x.size))
sol = solve_bvp(f, bc, x, u)
u_sol = sol.y[0]
plt.scatter(x, u_sol)

print("x.shape: {}, u.shape: {}, u_sol.shape: {}".format(x.shape, u.shape, u_sol.shape))

## Example: 1-D Heat Conduction

$$ \frac{d^2u}{dx^2} = 0 $$
$$ u(0) = T_0, u(1) = T_1 $$
$$ 0<x<1 $$

Step 1: Introduce substitution variable
    
$$ \frac{du}{dx} = v $$

Step 2: Re-write original equation
    
$$ \frac{dv}{dx} = 0 $$

Step 3: Re-write boundary conditions

$$ u(0) = T_0, u(1) = T_1 $$

Step 4: Combine previous steps

$$ \frac{du}{dx} = v $$
$$ \frac{dv}{dx} = 0 $$
$$ u(0) = T_0, u(1) = T_1 $$

In [ ]:
T0 = 100.
T1 = 275.

def f(x, u):
    retval = np.stack((u[1], np.zeros(x.size)), axis=0)
    print("retval.shape: {}".format(retval.shape))
    return retval

def bc(ua, ub):
    print("ua.shape: {}, ub.shape: {}".format(ua.shape, ub.shape))
    return [ua[0]-T0, ub[0]-T1]

x = np.linspace(0, 1)
u = np.zeros((2, x.size))
sol = solve_bvp(f, bc, x, u)
u_sol = sol.y[0]
plt.scatter(x, u_sol)

## Exercise: Plane Poiseuille Flow

Plane Poiseuille flow is flow created between two infinitely long parallel plates, separated by a distance $h$ with a constant pressure gradient ($ G = − \frac{dp}{dx} = constant $) applied in the direction of flow. The flow is essentially unidirectional because of infinite length. We can write the Navier-Stokes equation as:

$$ \frac{d^2u}{dx^2} = - \frac{G}{\mu} $$

and the no-slip conditions as:

$$ u(0) = 0, u(h) = 0 $$

Solve the equation given:

$$ h = 1, \frac{G}{\mu} = 1.0 $$

## Example: 1-D Reaction-diffusion equation

The one-dimensional, time-dependent reaction-diffusion equation can be written as:
    
$$ \frac{\partial c}{\partial t} = \mathscr{D} \frac{\partial^2 c}{\partial x^2} + R(c) $$

where $\mathscr{D}$ is the diffusion constant and $R(c)$ is the rate of generation by chemical reaction.

We are interested in the steady-state concentration profile given a first order reaction and the following boundary conditions:

$$ R(c) = -k c $$
$$ c(0) = C_0, c(L) = C_0 $$
$$ 0 < x < L $$

We re-write the reaction-diffusion equation as:

$$ \frac{d^2u}{d\xi^2} = Da*u $$

Where we have introduce the following dimensionless variables:

$$ u = \frac{c}{C_0} $$
$$ \xi = \frac{x}{L} $$
$$ Da = \frac{k C_0^{n-1}}{\mathscr{D}L^2} $$

$Da$ is known as the Damköhler number, which describes the relative ratio between the chemical reaction rate (of order $n$) and mass transfer rate.

Finally, we re-write this as a system of coupled, first-order ODEs:

$$ \frac{du}{d\xi} = v $$
$$ \frac{dv}{d\xi} = Da*u $$
$$ u(0) = 1, u(1) = 1 $$
$$ 0 < x < 1 $$

In [ ]:
# Let's solve the equation for a few different values of Da
for Da in [1e-2, 1e-1, 1., 10.]:

    def f(x, y):
        return np.stack((y[1], y[0]*Da), axis=0)

    def bc(ya, yb):
        return [ya[0]-1, yb[0]-1]

    x = np.linspace(0, 1)
    u = np.zeros((2, x.size))
    sol = solve_bvp(f, bc, x, u)
    u_sol = sol.y[0]
    plt.scatter(x, u_sol, label="Da: {}".format(Da))

plt.xlabel("$\\xi=x/L$")
plt.ylabel("$u=c/C_0$")
plt.legend()
plt.show()
plt.close()

## Exercise: 1-D Reaction-diffusion equation

Try repeating the above analysis for a second-order reaction:
    
$$ R(c) = k c^2 $$